In [1]:
import numpy as np
import zarr
import os
from skimage.data import binary_blobs
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

In [2]:
path = r"omezarr/test_ngff_image.zarr"
os.mkdir(path)

mean_val=10
size_xy = 128
size_z = 10
rng = np.random.default_rng(0)
data = rng.poisson(mean_val, size=(size_z, size_xy, size_xy)).astype(np.uint8)

# write the image data
store = parse_url(path, mode="w").store
root = zarr.group(store=store)
write_image(image=data, group=root, axes="zyx", storage_options=dict(chunks=(1, size_xy, size_xy)))
# optional rendering settings
root.attrs["omero"] = {
    "channels": [{
        "color": "00FFFF",
        "window": {"start": 0, "end": 20, "min": 0, "max": 255},
        "label": "random",
        "active": True,
    }]
}


# add labels...
blobs = binary_blobs(length=size_xy, volume_fraction=0.1, n_dim=3).astype('int8')
blobs2 = binary_blobs(length=size_xy, volume_fraction=0.1, n_dim=3).astype('int8')
# blobs will contain values of 1, 2 and 0 (background)
blobs += 2 * blobs2

# label.shape is (size_xy, size_xy, size_xy), Slice to match the data
label = blobs[:size_z, :, :]

# write the labels to /labels
labels_grp = root.create_group("labels")
# the 'labels' .zattrs lists the named labels data
label_name = "blobs"
labels_grp.attrs["labels"] = [label_name]
label_grp = labels_grp.create_group(label_name)
# need 'image-label' attr to be recognized as label
label_grp.attrs["image-label"] = {
    "colors": [
        {"label-value": 1, "rgba": [255, 0, 0, 255]},
        {"label-value": 2, "rgba": [0, 255, 0, 255]},
        {"label-value": 3, "rgba": [255, 255, 0, 255]}
    ]
}

write_image(label, label_grp, axes="zyx")

[]

In [3]:
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
import napari

url = "https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.4/idr0062A/6001240.zarr"

# read the image data
store = parse_url(url, mode="r").store

reader = Reader(parse_url(url))
# nodes may include images, labels etc
nodes = list(reader())
# first node will be the image pixel data
image_node = nodes[0]

dask_data = image_node.data


In [4]:
# We can view this in napari
# NB: image axes are CZYX: split channels by C axis=0
viewer = napari.view_image(dask_data, channel_axis=0)
if __name__ == '__main__':
    napari.run()

ImportError: No Qt bindings could be found.

napari requires either PyQt5 or PySide2 to be installed in the environment.
To install the default backend (currently PyQt5), run "pip install napari[all]" 
You may also use "pip install napari[pyside2]"for Pyside2, or "pip install napari[pyqt5]" for PyQt5